In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchio as tio
import torchvision.transforms as transforms


from modules import Evaluator3D, MRIDataset, MRIDatasets, UNet3D
from modules.DataAugmentations import *
from modules.LossFunctions import DC_and_CE_loss, GDiceLossV2
from modules.Utils import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# There are multiple data folders belong to same dataset. Each of them processed in different ways.
# Therefore, path of the data and its name explicitly are defined.
dataset_ = MRIDatasets.dHCP
dataset_path = "data2\\rel3_dhcp_anat_pipeline\\"
cv_ = "cv3" # 5-fold cross-validation. Folds [cv1-cv5]

# Transformations.
transform_eval = None #transforms.Compose([RandomMotion(), RandomAffine(degrees=[15])])


# Split dataset.
val = MRIDataset(dataset_, "val", dataset_path, transform=transform_eval)
test = MRIDataset(dataset_, "test", dataset_path, transform=transform_eval)

torch.manual_seed(0)
val_loader = torch.utils.data.DataLoader(dataset=val, batch_size=1)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=1)

In [ ]:
model = UNet3D().to(device)
criterion = DC_and_CE_loss({'batch_dice': True, 'smooth': 1e-5, 'do_bg': False, 'square': False}, {})
model.load_state_dict(torch.load("output/UNet3D/Iteration7/run2/weights/70_model.pth"))

In [ ]:
tissue_classes = ["Background", "eCSF", "Gray Matter", "White Matter", "Ventricles", 
                  "Cerrebilium", "Deep Gray Matter", "Brain Stem"]

# Evaluate the last model on validation set.
evaluator = Evaluator3D(criterion, model, (128, 128, 128), val_loader)
val_loss, val_scores = evaluator.evaluate()
avg_val_scores = sum(val_scores) / len(val_scores)
# Convert Tensors to list.
val_scores = [score.tolist() for score in val_scores]
# Combine results and subject information to examine data carefully. 
val_results = pd.DataFrame(val_scores, index=val.meta_data["participant_id"], columns=tissue_classes)
val_results.drop(columns="Background", inplace=True)
val_results = pd.merge(val.meta_data, val_results, on=["participant_id"])

# Display results.
print(f"Average Validation Dice Scores{avg_val_scores}")
plt.boxplot(val_results.iloc[:, 3:]) # Plot only dice scoress in box plot.
plt.show()
val_results

In [ ]:
# Evaluate the last model on validation set. 
# If cross-validation was used test set is not available for evaluation.
evaluator = Evaluator3D(criterion, model, (128, 128, 128), test_loader)
test_loss, test_scores = evaluator.evaluate()
avg_test_scores = sum(test_scores) / len(test_scores)
# Convert Tensors to list.
test_scores = [score.tolist() for score in test_scores]
# Combine results and subject information to examine data carefully. 
test_results = pd.DataFrame(test_scores, index=test.meta_data["participant_id"], columns=tissue_classes)
test_results.drop(columns="Background", inplace=True)
test_results = pd.merge(test.meta_data, test_results, on=["participant_id"])

# Display results.
print(f"Average Test Dice Scores{avg_test_scores}")
plt.boxplot(test_results.iloc[:, 3:])
plt.show()
test_results